In [1]:
import json
import pandas as pd
import warnings
import re
import pickle

warnings.filterwarnings(action='ignore')

In [2]:
raw_path = "../data/raw_data/haystack.json"

with open(raw_path, "r") as json_file:
    json_data = json.load(json_file)

## 데이터 로드 후 [MRC](https://github.com/boostcampaitech3/level2-mrc-level2-nlp-09)데이터 형식과 같게 만들기 
- 데이터 쪼개기 (context, question, document_id, answers [{'answer_start', 'text'}] )
- Question 별로 유형 분류

In [3]:
json_data['data'][0]

{'paragraphs': [{'qas': [{'question': '1_제178회 완주군의회 임시회 회기는 언제로 결정됐어?',
     'id': 341139,
     'answers': [{'answer_id': 362212,
       'document_id': 603489,
       'question_id': 341139,
       'text': '4월 20일부터 4월 27일까지 8일간',
       'answer_start': 144,
       'answer_end': 165,
       'answer_category': None}],
     'is_impossible': False},
    {'question': '3_제178회 완주군의회 임시회 회기를 발의한 사람은 누구인가?',
     'id': 341140,
     'answers': [{'answer_id': 362213,
       'document_id': 603489,
       'question_id': 341140,
       'text': '김상식 위원장 외 3인의 의원',
       'answer_start': 119,
       'answer_end': 135,
       'answer_category': None}],
     'is_impossible': False}],
   'context': '@제178회        완주군의회(임시회) 제 1 차 본회의회의록@\n\n의사일정 제1항 제178회 완주군의회 임시회 회기결정의 건을 상정합니다.  제178회 완주군의회 임시회 회기 결정의 건에 대하여는 운영위원회 김상식 위원장 외 3인의 의원이 발의 한대로 4월 20일부터 4월 27일까지 8일간으로 회기를 결정하고자 합니다. 의원 여러분 이의 있으십니까?\n\n(『없습니다』하는 의원 있음)\n\n이의가 없으므로 가결되었음을 선포합니다.\n',
   'document_id': 603489}]}

In [4]:
def answer_pick(answers):
    dict_answer = {"answer_start": answers['answer_start'],
                   "text": answers['text']
                  }
    return dict_answer

In [5]:
num_ls = [str(i) for i in range(1, 8)]

dict_data = []
no_category = []
# json parsing
for i, v in enumerate(json_data['data']):
    content = v['paragraphs'][0]
    for qas_dicts in content['qas']:
        qas_dict = qas_dicts.copy()
        qas_dict['context'] = content['context']
        qas_dict['document_id'] = content['document_id']
        qas_dict['answers'] = answer_pick(qas_dict['answers'][0])
        if len(qas_dict['question'].split('_')[0]) > 1:
            no_category.append(qas_dict)
        else:
            dict_data.append(qas_dict)

## category 있는 애들
- 데이터 태깅할 때, (질문유형번호)_(질문) 으로 태깅
- Ex : 3_~한 사람은 누구인가?

In [6]:
df = pd.DataFrame(dict_data)

In [7]:
 # Dataframe parsing / category split
df['category'] = df.question.str.split('_').str[0]
df['question'] = df.question.str.split('_').str[1]
df.head()

,question,id,answers,is_impossible,context,document_id,category
0,제178회 완주군의회 임시회 회기는 언제로 결정됐어?,341139,"{'answer_start': 144, 'text': '4월 20일부터 4월 27일...",False,@제178회 완주군의회(임시회) 제 1 차 본회의회의록@\n\n의사일정...,603489,1
1,제178회 완주군의회 임시회 회기를 발의한 사람은 누구인가?,341140,"{'answer_start': 119, 'text': '김상식 위원장 외 3인의 의원'}",False,@제178회 완주군의회(임시회) 제 1 차 본회의회의록@\n\n의사일정...,603489,3
2,제7대 완주군의회 후반기 상임위원회 의원 선임건을 상정한 회의는 어떤거야?,341040,"{'answer_start': 1, 'text': '제214회 완주군의회(임시회) ...",False,@제214회 완주군의회(임시회) 제1차 본 회 의 회 의 록@\n\n의석을 정돈하여...,604003,7
3,산업건설위원회 위원들은 누구인가?,341275,"{'answer_start': 656, 'text': '서남용 의원님, 윤수봉 의원...",False,@제214회 완주군의회(임시회) 제1차 본 회 의 회 의 록@\n\n의석을 정돈하여...,604003,4
4,완주군의회 의원 의정활동비등 지급에 관한 조례중 개정조례안을 발의한 의원은 몇 명이야?,341034,"{'answer_start': 435, 'text': '9인'}",False,@제109회 전라북도 완주군의회 (임시회) 제2호 본 회 의 회 의 록@\n\n의사...,603644,6


In [8]:
# reorder
data_df = df[['category', 'document_id', 'question', 'answers', 'context']]

In [9]:
data_df = data_df.astype({'category':'int'})

## 카테고리 없는 애들
- 33개 데이터 카테고리 미태깅, 수가 얼마 안돼 카테고리 태깅만 해서 넣어줌

In [10]:
df_no_cat = pd.DataFrame(no_category)

In [11]:
category = [2, 1, 3, 6, 4, 5, 2, 1, 5, 3, 7, 4, 1, 2, 4, 2, 2, 3, 2, 4, 6, 5, 6, 6, 5, 5, 7, 7, 4, 2, 4, 2, 7, 4]

In [12]:
df_no_cat['category'] = category

In [13]:
data_df_no_cat = df_no_cat[['category', 'document_id', 'question', 'answers', 'context']]
# 기존 데이터 폴더에서 더함

In [14]:
data_concat = pd.concat([data_df, data_df_no_cat])

In [15]:
data_concat

,category,document_id,question,answers,context
0,1,603489,제178회 완주군의회 임시회 회기는 언제로 결정됐어?,"{'answer_start': 144, 'text': '4월 20일부터 4월 27일...",@제178회 완주군의회(임시회) 제 1 차 본회의회의록@\n\n의사일정...
1,3,603489,제178회 완주군의회 임시회 회기를 발의한 사람은 누구인가?,"{'answer_start': 119, 'text': '김상식 위원장 외 3인의 의원'}",@제178회 완주군의회(임시회) 제 1 차 본회의회의록@\n\n의사일정...
2,7,604003,제7대 완주군의회 후반기 상임위원회 의원 선임건을 상정한 회의는 어떤거야?,"{'answer_start': 1, 'text': '제214회 완주군의회(임시회) ...",@제214회 완주군의회(임시회) 제1차 본 회 의 회 의 록@\n\n의석을 정돈하여...
3,4,604003,산업건설위원회 위원들은 누구인가?,"{'answer_start': 656, 'text': '서남용 의원님, 윤수봉 의원...",@제214회 완주군의회(임시회) 제1차 본 회 의 회 의 록@\n\n의석을 정돈하여...
4,6,603644,완주군의회 의원 의정활동비등 지급에 관한 조례중 개정조례안을 발의한 의원은 몇 명이야?,"{'answer_start': 435, 'text': '9인'}",@제109회 전라북도 완주군의회 (임시회) 제2호 본 회 의 회 의 록@\n\n의사...
...,...,...,...,...,...
29,2,604280,제62회 전라북도 완주군의회 2차 본회의는 언제야?,"{'answer_start': 267, 'text': '7월 11일 오전 10시'}",@제62회 전라북도 완주군의회 (임시회) 제1호 본 회 의 회 의 록@\n\n다음은...
30,4,604312,제173회 임시회 회의록 서명의원은 누가 맡기로 했지?,"{'answer_start': 107, 'text': '정성모 의원님과 김상식 의원님'}",@제173회 완주군의회(임시회) 제 1 차 본회의회의록@\n\n의사일정 ...
31,2,604331,제204회 완주군의회 2차 본회의는 언제야?,"{'answer_start': 217, 'text': '4월 28일 오전 10시'}",@제204회 완주군의회(임시회) 제 1 차 본회의회의록@\n\n다음은 본회의 휴회의...
32,7,603661,93년도 완주군 상수도 급수조례중 개정조례안을 상정한 회의는?,"{'answer_start': 1, 'text': '제24회 전라북도 완주군의회(정...",@제24회 전라북도 완주군의회(정기회) 제6호 본 회 의 회 의 록@\n\n다음은 ...


In [16]:
## index 맞추기 위해 다시 저장 
data_concat.to_csv("../data/raw_data/origin_data.csv", mode='w', index=False)

In [17]:
df=pd.read_csv('../data/raw_data/origin_data.csv')

# Mislabeling 수정& 전처리

## question null 값
- 486 -> 질문이 '2' ->직접 질문 넣음
- 799 -> 질문이 NaN -> 직접 질문넣음
- 1396 -> 질문이 '3' -> 직접 질문넣음 

In [18]:
tmp1=df.iloc[486]
tmp2=df.iloc[799]
tmp3=df.iloc[1396]

tmp1['question']='제240회 완주군의회 임시회에서 의회 관련 사항을 보고한 사람은 누구야?'
tmp2['question']='제98회 전라북도 완주군의회 임시회 회기 결정의 건에 대해 발의한 사람은 누구야?'
tmp3['question']='제 103회 전라북도 완주군의회 임시회 제1회 본회의의 휴회 기간은 언제야?'

df.iloc[486]=tmp1
df.iloc[799]=tmp2
df.iloc[1396]=tmp3

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1553 entries, 0 to 1552
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   category     1553 non-null   int64 
 1   document_id  1553 non-null   int64 
 2   question     1553 non-null   object
 3   answers      1553 non-null   object
 4   context      1553 non-null   object
dtypes: int64(2), object(3)
memory usage: 60.8+ KB


# question 내의 "\n" 제거 ->10개당 1개 꼴


In [19]:
def remove_n(text):
    new_string = re.sub(r'\n', '', text)
    return new_string

df['question']=df['question'].apply(lambda x: remove_n(x))

## 태깅이 잘못되어 있는 것들 확인 -> 13개 제거
- Tokenize 이후 확인 
- Ex : '나는 이재학이다' ->  '는 이재학이다.'로 태깅한 경우

In [20]:
df=df.drop([84,117,387,464,546,644,683,750,812,991,1057,1426,1460])

In [21]:
# index 맞추기 위해 저장
df.to_csv("../data/raw_data/origin_data.csv", mode='w', index=False)
df = pd.read_csv("../data/raw_data/origin_data.csv")

## No_answer : @ -> 공백

In [22]:
# dict 자료형인 answers 열이 string으로 불러와지는 것 해결
df['answers']=df['answers'].apply(lambda x:eval(x))

for i in range(len(df)):
    if len(df['answers'][i]['text'])==1 and df['answers'][i]['text']=="@":
        df['answers'][i]['text']=" "

## answer text의 좌우공백 제거
- start index의 경우 공백의 개수만큼 줄이기
- finish index의 경우 text의 길이만큼 더해주니, 우측 공백만 제거
- 145개의 데이터 수정

In [23]:
answer_list=df['answers']

for i in range(len(answer_list)):
    count=0
    if answer_list[i]['text']!=" " and len(answer_list[i]['text'])> 1 :
        for j in range(len(list(answer_list[i]['text']))):
            if list(answer_list[i]['text'])[j]==" ":
                count+=1
            if list(answer_list[i]['text'])[j+1]!=" ":
                break
        answer_list[i]['answer_start']=answer_list[i]['answer_start']-count
        answer_list[i]['text']=answer_list[i]['text'].strip()
    
df['answers']=answer_list

# answer text의 이상치 제거
- 태깅을 잘못한 경우에 해당
- 답변의 길이의 평균:16, 제3사분위수:23인 이지만, 1376의 길이를 갖음 -> 본문전체를 태깅함
- row 하나 제거

In [24]:
df['ans_len']=[len((df['answers'][i])['text']) for i in range(len(df))]
df.sort_values('ans_len',ascending=False).head(3)

,category,document_id,question,answers,context,ans_len
534,3,603665,제117회 전라북도 완주군의회 제2차 정례회에서 2005년도 군정방향에 대한 설명을...,"{'answer_start': 765, 'text': '내년도 군정 여건을 살펴보면...",@제117회 전라북도 완주군의회 (제2차 정례회) 제1호 본 회 의 회 의 록@\n...,1367
738,5,603843,제113회 완주군의회 1차 정례회가 7월 6일에 개회된 이유는 뭐야?,"{'answer_start': 168, 'text': '완주군의회 정례회의 운영에 ...",@제113회 전라북도 완주군의회(1차 정례회) 제1호 본 회 의 회 의 록@\n\n...,154
823,5,603910,제 149회 전라북도 완주군의회 제2차 정례회 제 1호 본회의에서 다룬 군수 및 관...,"{'answer_start': 223, 'text': '군민의 뜻과 목소리가 군정에...",@제149회 전라북도 완주군의회(제2차 정례회) 제1호 본 회 의 회 의 록@\n\...,119


In [25]:
df=df.drop([534])

## csv pickle로 저장
- 데이터프레임 안의 dict 타입 보존 안되고 str 타입으로 나오는 문제 -> pickle로 해결 가능
- pickle의 경우 데이터를 빠르게 읽고 불러올 수 있음
    - [참고](https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=wideeyed&logNo=221250772912)
- pickle 저장하고, pd.read_pickle 할 때 pkl 파일 확인 후 불러와야 함

In [26]:
df.to_pickle("../data/raw_data/origin_data.pkl")
df.to_csv("../data/raw_data/origin_data.csv", mode='w', index=False)

# csv 파일 불러오기

In [27]:
final_csv = pd.read_pickle("../data/raw_data/origin_data.pkl")

In [28]:
final_csv.head(5)

,category,document_id,question,answers,context,ans_len
0,1,603489,제178회 완주군의회 임시회 회기는 언제로 결정됐어?,"{'answer_start': 144, 'text': '4월 20일부터 4월 27일...",@제178회 완주군의회(임시회) 제 1 차 본회의회의록@\n\n의사일정...,21
1,3,603489,제178회 완주군의회 임시회 회기를 발의한 사람은 누구인가?,"{'answer_start': 119, 'text': '김상식 위원장 외 3인의 의원'}",@제178회 완주군의회(임시회) 제 1 차 본회의회의록@\n\n의사일정...,16
2,7,604003,제7대 완주군의회 후반기 상임위원회 의원 선임건을 상정한 회의는 어떤거야?,"{'answer_start': 1, 'text': '제214회 완주군의회(임시회) ...",@제214회 완주군의회(임시회) 제1차 본 회 의 회 의 록@\n\n의석을 정돈하여...,26
3,4,604003,산업건설위원회 위원들은 누구인가?,"{'answer_start': 656, 'text': '서남용 의원님, 윤수봉 의원...",@제214회 완주군의회(임시회) 제1차 본 회 의 회 의 록@\n\n의석을 정돈하여...,43
4,6,603644,완주군의회 의원 의정활동비등 지급에 관한 조례중 개정조례안을 발의한 의원은 몇 명이야?,"{'answer_start': 435, 'text': '9인'}",@제109회 전라북도 완주군의회 (임시회) 제2호 본 회 의 회 의 록@\n\n의사...,2


In [29]:
type(final_csv['answers'][0])

dict